In [1]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    DataCollatorWithPadding,
    Trainer,
)
from datasets import load_dataset

d:\Internship Tasks\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("ag_news")

In [3]:
print(dataset["train"][0])

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'label': 2}


In [4]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)


def Tokenizer(dataset):
    return tokenizer(
        dataset["text"], truncation=True, padding="max_length", max_length=128
    )


tokenized_datasets = dataset.map(Tokenizer, batched=True)


In [5]:
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
print(tokenized_datasets["train"][0].keys())

dict_keys(['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'])


In [6]:
tokenized_datasets.set_format(
    "torch", columns=["input_ids", "attention_mask", "labels"]
)
print(tokenized_datasets["train"][0].keys())

dict_keys(['labels', 'input_ids', 'attention_mask'])


In [7]:
labels = 4
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=labels
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
training_args = TrainingArguments(
    output_dir="./ag_news_bert_results",  # output directory
    eval_strategy="epoch",  # evaluate at the end of each epoch (updated parameter name)
    save_strategy="epoch",  # save at the end of each epoch
    learning_rate=2e-5,  # learning rate
    per_device_train_batch_size=16,  # batch size for training
    per_device_eval_batch_size=16,  # batch size for evaluation
    num_train_epochs=3,  # number of training epochs
    weight_decay=0.01,  # strength of weight decay
    load_best_model_at_end=True,  # load the best model at the end
    metric_for_best_model="accuracy",  # metric to choose the best model
)

In [9]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="weighted")
    return {"accuracy": acc, "f1_score": f1}

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

C:\Users\HP\AppData\Local\Temp\ipykernel_12136\4052521561.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
d:\Internship Tasks\.venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


In [ ]:
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

In [ ]:
# Save the model and tokenizer
model.save_pretrained('./news_classifier')
tokenizer.save_pretrained('./news_classifier')